# 09_ModelSelection - Physics-SR Framework v3.0

## Stage 3.1: Model Selection via K-Fold CV and EBIC

**Author:** Zhengze Zhang  
**Affiliation:** Department of Statistics, Columbia University  
**Date:** January 2026

---

### Purpose

Compare and select the best model from Stage 2 candidates (PySR, E-WSINDy, Adaptive Lasso) using:
1. **K-Fold Cross-Validation** for generalization assessment
2. **Extended BIC (EBIC)** for high-dimensional model selection

### K-Fold Cross-Validation

Robust model comparison that estimates out-of-sample performance:
- Split data into K folds
- Train on K-1 folds, test on held-out fold
- Repeat K times, average results

### Extended BIC (EBIC)

High-dimensional model selection with proper penalty for large search spaces:
$$EBIC_\gamma = n \cdot \log(RSS/n) + k \cdot \log(n) + 2\gamma \cdot \log\binom{p}{k}$$

where:
- $n$ = sample size
- $k$ = number of selected terms
- $p$ = total features in library
- $\gamma \in [0, 1]$ = tuning parameter

### Reference

- Chen, J., & Chen, Z. (2008). Extended Bayesian information criteria for model selection with large model spaces. *Biometrika*, 95(3), 759-771.

---
## Section 1: Header and Imports

In [ ]:
"""
09_ModelSelection.ipynb - Model Selection via CV and EBIC
==========================================================

Three-Stage Physics-Informed Symbolic Regression Framework v3.0

This module provides:
- ModelSelector: Compare models via K-fold CV and EBIC
- K-fold cross-validation for generalization assessment
- Extended BIC for high-dimensional model selection
- One-SE rule for parsimony preference

Algorithm:
    1. Collect candidate models from Stage 2
    2. Evaluate each via K-fold CV (MSE, R2)
    3. Compute EBIC scores for model ranking
    4. Select best model (optionally with one-SE rule)

Author: Zhengze Zhang
Affiliation: Department of Statistics, Columbia University
"""

# Import core module
%run 00_Core.ipynb

In [ ]:
# Additional imports for Model Selection
from sklearn.model_selection import KFold
from scipy.special import comb
from typing import Dict, List, Tuple, Optional, Any

print("09_ModelSelection: Additional imports successful.")

---
## Section 2: Class Definition

In [ ]:
# ==============================================================================
# MODEL SELECTOR CLASS
# ==============================================================================

class ModelSelector:
    """
    Model Selection via K-Fold Cross-Validation and EBIC.
    
    Compares candidate models from Stage 2 (PySR, E-WSINDy, Adaptive Lasso)
    using multiple selection criteria:
    - K-fold CV for generalization assessment
    - EBIC for high-dimensional model selection
    - One-SE rule for parsimony preference
    
    Attributes
    ----------
    n_folds : int
        Number of cross-validation folds (default: 5)
    ebic_gamma : float
        EBIC parameter in [0, 1] (default: 0.5)
        Higher gamma = more penalty for complex models
    
    Examples
    --------
    >>> selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
    >>> result = selector.compare_models(candidates, y)
    >>> print(f"Best model: {result['best_model_cv']}")
    """
    
    def __init__(
        self,
        n_folds: int = DEFAULT_CV_FOLDS,
        ebic_gamma: float = DEFAULT_EBIC_GAMMA,
        random_state: int = RANDOM_SEED
    ):
        """
        Initialize ModelSelector.
        
        Parameters
        ----------
        n_folds : int
            Number of CV folds. Default: 5
        ebic_gamma : float
            EBIC parameter. 0.0 = BIC, 0.5 = balanced, 1.0 = max penalty.
            Default: 0.5
        random_state : int
            Random seed for CV splits. Default: 42
        """
        self.n_folds = n_folds
        self.ebic_gamma = ebic_gamma
        self.random_state = random_state
        
        # Internal state
        self._cv_results = None
        self._ebic_results = None
        self._best_model_cv = None
        self._best_model_ebic = None
        self._best_model_onese = None
        self._comparison_complete = False
    
    def compare_models(
        self,
        candidates: Dict[str, Tuple[np.ndarray, np.ndarray]],
        y: np.ndarray,
        p_total: int = None
    ) -> Dict[str, Any]:
        """
        Compare candidate models using CV and EBIC.
        
        Parameters
        ----------
        candidates : Dict[str, Tuple[np.ndarray, np.ndarray]]
            Dictionary mapping model name to (Phi, support) tuple:
            - Phi: Feature matrix used by the model
            - support: Boolean mask of selected features
        y : np.ndarray
            Target vector
        p_total : int, optional
            Total number of features in full library (for EBIC).
            If None, uses max Phi.shape[1] across candidates.
        
        Returns
        -------
        Dict[str, Any]
            Dictionary containing:
            - cv_results: Dict of model -> (cv_mean, cv_std, cv_scores)
            - ebic_results: Dict of model -> ebic_score
            - best_model_cv: Name of best model by CV
            - best_model_ebic: Name of best model by EBIC
            - best_model_onese: Name of best model by one-SE rule
            - ranking_cv: Models ranked by CV performance
            - ranking_ebic: Models ranked by EBIC
        """
        if p_total is None:
            p_total = max(Phi.shape[1] for Phi, _ in candidates.values())
        
        # K-fold CV evaluation
        self._cv_results = {}
        for name, (Phi, support) in candidates.items():
            cv_mean, cv_std, cv_scores = self._kfold_cv(Phi, y, support)
            self._cv_results[name] = {
                'cv_mean': cv_mean,
                'cv_std': cv_std,
                'cv_scores': cv_scores,
                'cv_se': cv_std / np.sqrt(self.n_folds)
            }
        
        # EBIC evaluation
        self._ebic_results = {}
        for name, (Phi, support) in candidates.items():
            ebic = self._compute_ebic(Phi, y, support, p_total)
            self._ebic_results[name] = ebic
        
        # Select best models
        self._best_model_cv = min(
            self._cv_results.keys(),
            key=lambda x: self._cv_results[x]['cv_mean']
        )
        
        self._best_model_ebic = min(
            self._ebic_results.keys(),
            key=lambda x: self._ebic_results[x]
        )
        
        # One-SE rule
        self._best_model_onese = self._one_se_rule(candidates)
        
        # Rankings
        ranking_cv = sorted(
            self._cv_results.keys(),
            key=lambda x: self._cv_results[x]['cv_mean']
        )
        
        ranking_ebic = sorted(
            self._ebic_results.keys(),
            key=lambda x: self._ebic_results[x]
        )
        
        self._comparison_complete = True
        
        return {
            'cv_results': self._cv_results,
            'ebic_results': self._ebic_results,
            'best_model_cv': self._best_model_cv,
            'best_model_ebic': self._best_model_ebic,
            'best_model_onese': self._best_model_onese,
            'ranking_cv': ranking_cv,
            'ranking_ebic': ranking_ebic,
            'n_folds': self.n_folds,
            'ebic_gamma': self.ebic_gamma
        }
    
    def _kfold_cv(
        self,
        Phi: np.ndarray,
        y: np.ndarray,
        support: np.ndarray
    ) -> Tuple[float, float, np.ndarray]:
        """
        Perform K-fold cross-validation.
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature matrix
        y : np.ndarray
            Target vector
        support : np.ndarray
            Boolean mask of selected features
        
        Returns
        -------
        Tuple[float, float, np.ndarray]
            (cv_mean_mse, cv_std_mse, fold_scores)
        """
        # Use only supported features
        Phi_support = Phi[:, support]
        
        if Phi_support.shape[1] == 0:
            # No features selected - return high error
            return np.inf, 0.0, np.full(self.n_folds, np.inf)
        
        kf = KFold(
            n_splits=self.n_folds,
            shuffle=True,
            random_state=self.random_state
        )
        
        fold_scores = []
        
        for train_idx, test_idx in kf.split(Phi_support):
            # Split data
            X_train = Phi_support[train_idx]
            X_test = Phi_support[test_idx]
            y_train = y[train_idx]
            y_test = y[test_idx]
            
            # Fit OLS on training data
            try:
                beta, _, _, _ = np.linalg.lstsq(X_train, y_train, rcond=None)
            except np.linalg.LinAlgError:
                fold_scores.append(np.inf)
                continue
            
            # Predict on test data
            y_pred = X_test @ beta
            
            # Compute MSE
            mse = np.mean((y_test - y_pred)**2)
            fold_scores.append(mse)
        
        fold_scores = np.array(fold_scores)
        return np.mean(fold_scores), np.std(fold_scores), fold_scores
    
    def _compute_ebic(
        self,
        Phi: np.ndarray,
        y: np.ndarray,
        support: np.ndarray,
        p_total: int
    ) -> float:
        """
        Compute Extended BIC score.
        
        EBIC_gamma = n * log(RSS/n) + k * log(n) + 2 * gamma * log(C(p,k))
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature matrix
        y : np.ndarray
            Target vector
        support : np.ndarray
            Boolean mask of selected features
        p_total : int
            Total number of features in library
        
        Returns
        -------
        float
            EBIC score (lower is better)
        """
        n = len(y)
        k = int(np.sum(support))
        
        if k == 0:
            return np.inf
        
        # Fit OLS on support
        Phi_support = Phi[:, support]
        try:
            beta, _, _, _ = np.linalg.lstsq(Phi_support, y, rcond=None)
        except np.linalg.LinAlgError:
            return np.inf
        
        # Compute RSS
        y_pred = Phi_support @ beta
        rss = np.sum((y - y_pred)**2)
        
        if rss <= 0:
            rss = EPS_DIV
        
        # EBIC formula
        # log(RSS/n) term
        log_likelihood_term = n * np.log(rss / n)
        
        # BIC penalty: k * log(n)
        bic_penalty = k * np.log(n)
        
        # Extended penalty: 2 * gamma * log(C(p,k))
        # Use log of combination to avoid overflow
        if k <= p_total:
            log_comb = self._log_comb(p_total, k)
            extended_penalty = 2 * self.ebic_gamma * log_comb
        else:
            extended_penalty = 0
        
        ebic = log_likelihood_term + bic_penalty + extended_penalty
        
        return ebic
    
    def _log_comb(
        self,
        n: int,
        k: int
    ) -> float:
        """
        Compute log of binomial coefficient.
        
        log(C(n,k)) = log(n!) - log(k!) - log((n-k)!)
        Uses Stirling approximation for large values.
        
        Parameters
        ----------
        n : int
            Total items
        k : int
            Items to choose
        
        Returns
        -------
        float
            log(C(n,k))
        """
        if k == 0 or k == n:
            return 0.0
        if k > n:
            return 0.0
        
        # Use scipy's log of gamma function for accuracy
        from scipy.special import gammaln
        return gammaln(n + 1) - gammaln(k + 1) - gammaln(n - k + 1)
    
    def _one_se_rule(
        self,
        candidates: Dict[str, Tuple[np.ndarray, np.ndarray]]
    ) -> str:
        """
        Apply one-SE rule for model selection.
        
        Select the simplest model within 1 standard error of the best.
        
        Parameters
        ----------
        candidates : Dict
            Candidate models
        
        Returns
        -------
        str
            Name of selected model
        """
        # Get best CV score and its SE
        best_cv_mean = self._cv_results[self._best_model_cv]['cv_mean']
        best_cv_se = self._cv_results[self._best_model_cv]['cv_se']
        
        # Threshold: best + 1 SE
        threshold = best_cv_mean + best_cv_se
        
        # Find simplest model within threshold
        eligible_models = []
        for name, (Phi, support) in candidates.items():
            if self._cv_results[name]['cv_mean'] <= threshold:
                complexity = int(np.sum(support))
                eligible_models.append((name, complexity))
        
        if not eligible_models:
            return self._best_model_cv
        
        # Select simplest
        return min(eligible_models, key=lambda x: x[1])[0]
    
    def get_best_model(
        self,
        criterion: str = 'cv'
    ) -> str:
        """
        Get best model by specified criterion.
        
        Parameters
        ----------
        criterion : str
            'cv', 'ebic', or 'onese'
        
        Returns
        -------
        str
            Name of best model
        """
        if not self._comparison_complete:
            raise ValueError("Must run compare_models() first")
        
        if criterion == 'cv':
            return self._best_model_cv
        elif criterion == 'ebic':
            return self._best_model_ebic
        elif criterion == 'onese':
            return self._best_model_onese
        else:
            raise ValueError(f"Unknown criterion: {criterion}")
    
    def print_comparison_report(self) -> None:
        """
        Print detailed model comparison report.
        """
        if not self._comparison_complete:
            print("Comparison not yet performed. Run compare_models() first.")
            return
        
        print("=" * 70)
        print(" Model Selection Results")
        print("=" * 70)
        print()
        print(f"Configuration:")
        print(f"  CV folds: {self.n_folds}")
        print(f"  EBIC gamma: {self.ebic_gamma}")
        print()
        print("-" * 70)
        print(" Cross-Validation Results:")
        print("-" * 70)
        print(f"{'Model':<25} {'CV Mean MSE':<15} {'CV Std':<12} {'CV SE':<12}")
        print("-" * 70)
        
        # Sort by CV mean
        sorted_models = sorted(
            self._cv_results.keys(),
            key=lambda x: self._cv_results[x]['cv_mean']
        )
        
        for name in sorted_models:
            r = self._cv_results[name]
            marker = " *" if name == self._best_model_cv else ""
            print(f"{name:<25} {r['cv_mean']:<15.6f} {r['cv_std']:<12.6f} "
                  f"{r['cv_se']:<12.6f}{marker}")
        
        print()
        print("-" * 70)
        print(" EBIC Results:")
        print("-" * 70)
        print(f"{'Model':<25} {'EBIC Score':<15}")
        print("-" * 45)
        
        sorted_ebic = sorted(
            self._ebic_results.keys(),
            key=lambda x: self._ebic_results[x]
        )
        
        for name in sorted_ebic:
            ebic = self._ebic_results[name]
            marker = " *" if name == self._best_model_ebic else ""
            print(f"{name:<25} {ebic:<15.4f}{marker}")
        
        print()
        print("-" * 70)
        print(" Best Model Selection:")
        print("-" * 70)
        print(f"  By CV:        {self._best_model_cv}")
        print(f"  By EBIC:      {self._best_model_ebic}")
        print(f"  By One-SE:    {self._best_model_onese}")
        print()
        print("=" * 70)

---
## Section 3: Internal Tests

In [ ]:
# ==============================================================================
# TEST CONTROL FLAG
# ==============================================================================

_RUN_TESTS = False  # Set to True to run internal tests

if _RUN_TESTS:
    print("=" * 70)
    print(" RUNNING INTERNAL TESTS FOR 09_ModelSelection")
    print("=" * 70)

In [ ]:
# ==============================================================================
# TEST 1: CV with Known Best Model
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 1: CV with Known Best Model")
    
    np.random.seed(42)
    n_samples = 200
    
    # Generate data
    x1 = np.random.randn(n_samples)
    x2 = np.random.randn(n_samples)
    x3 = np.random.randn(n_samples)
    
    # True: y = 2*x1 + x2
    y = 2*x1 + x2 + 0.1*np.random.randn(n_samples)
    
    # Feature library
    Phi = np.column_stack([np.ones(n_samples), x1, x2, x3, x1**2, x2**2])
    
    # Candidate models
    candidates = {
        'correct': (Phi, np.array([False, True, True, False, False, False])),  # x1, x2
        'overfit': (Phi, np.array([True, True, True, True, True, True])),       # All
        'underfit': (Phi, np.array([False, True, False, False, False, False])), # Only x1
    }
    
    print("True model: y = 2*x1 + x2")
    print("Candidates: correct (x1,x2), overfit (all), underfit (x1 only)")
    print()
    
    selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
    result = selector.compare_models(candidates, y, p_total=6)
    
    selector.print_comparison_report()
    
    if result['best_model_cv'] == 'correct':
        print("[PASS] CV correctly identified the true model")
    else:
        print(f"[INFO] CV selected: {result['best_model_cv']}")

In [ ]:
# ==============================================================================
# TEST 2: EBIC Penalty Verification
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 2: EBIC Penalty Verification")
    
    np.random.seed(42)
    n_samples = 100
    
    x1 = np.random.randn(n_samples)
    x2 = np.random.randn(n_samples)
    
    y = 2*x1 + 0.1*np.random.randn(n_samples)
    
    # Feature library with many features
    Phi = np.column_stack([x1, x2] + [np.random.randn(n_samples) for _ in range(18)])
    p_total = 20
    
    # Models of increasing complexity
    candidates = {
        'k=1': (Phi, np.array([True] + [False]*19)),
        'k=5': (Phi, np.array([True]*5 + [False]*15)),
        'k=10': (Phi, np.array([True]*10 + [False]*10)),
    }
    
    print(f"True model uses only x1 (k=1)")
    print(f"Testing EBIC penalty for k=1, 5, 10 with gamma=0.5")
    print()
    
    selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
    result = selector.compare_models(candidates, y, p_total=p_total)
    
    print(f"EBIC scores:")
    for name in ['k=1', 'k=5', 'k=10']:
        print(f"  {name}: {result['ebic_results'][name]:.2f}")
    
    # EBIC should prefer simpler model
    if result['best_model_ebic'] == 'k=1':
        print("\n[PASS] EBIC correctly penalized complex models")
    else:
        print(f"\n[INFO] EBIC selected: {result['best_model_ebic']}")

In [ ]:
# ==============================================================================
# TEST 3: One-SE Rule
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 3: One-SE Rule")
    
    np.random.seed(42)
    n_samples = 200
    
    x1 = np.random.randn(n_samples)
    x2 = np.random.randn(n_samples)
    x3 = np.random.randn(n_samples)
    
    # y = 2*x1 + 0.1*x2 (x2 has very small effect)
    y = 2*x1 + 0.1*x2 + 0.5*np.random.randn(n_samples)
    
    Phi = np.column_stack([x1, x2, x3])
    
    # Complex model slightly better but within SE
    candidates = {
        'simple': (Phi, np.array([True, False, False])),  # k=1
        'medium': (Phi, np.array([True, True, False])),   # k=2 (true)
        'complex': (Phi, np.array([True, True, True])),   # k=3
    }
    
    print("True: y = 2*x1 + 0.1*x2 (noisy)")
    print("One-SE rule should prefer simpler model if within 1 SE of best")
    print()
    
    selector = ModelSelector(n_folds=5)
    result = selector.compare_models(candidates, y)
    
    selector.print_comparison_report()
    
    print(f"\nOne-SE selection: {result['best_model_onese']}")
    print("(May prefer simpler model if CV scores are close)")

In [ ]:
# ==============================================================================
# TEST 4: EBIC Gamma Sensitivity
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 4: EBIC Gamma Sensitivity")
    
    np.random.seed(42)
    n_samples = 100
    
    x1 = np.random.randn(n_samples)
    y = 2*x1 + 0.1*np.random.randn(n_samples)
    
    # Large library
    Phi = np.column_stack([x1] + [np.random.randn(n_samples) for _ in range(49)])
    p_total = 50
    
    candidates = {
        'sparse': (Phi, np.array([True] + [False]*49)),
        'dense': (Phi, np.array([True]*10 + [False]*40)),
    }
    
    gamma_values = [0.0, 0.5, 1.0]
    
    print(f"Testing EBIC with different gamma values:")
    print(f"{'Gamma':<10} {'Sparse EBIC':<15} {'Dense EBIC':<15} {'Best'}")
    print("-" * 55)
    
    for gamma in gamma_values:
        selector = ModelSelector(ebic_gamma=gamma)
        result = selector.compare_models(candidates, y, p_total=p_total)
        
        print(f"{gamma:<10.1f} {result['ebic_results']['sparse']:<15.2f} "
              f"{result['ebic_results']['dense']:<15.2f} {result['best_model_ebic']}")
    
    print()
    print("Note: Higher gamma = stronger penalty for complex models")

---
## Section 4: Module Summary

In [ ]:
# ==============================================================================
# MODULE SUMMARY
# ==============================================================================

print("=" * 70)
print(" 09_ModelSelection.ipynb - Module Summary")
print("=" * 70)
print()
print("CLASS: ModelSelector")
print("-" * 70)
print()
print("Purpose:")
print("  Compare and select best model from Stage 2 candidates using")
print("  K-fold cross-validation and Extended BIC.")
print()
print("Main Methods:")
print("  compare_models(candidates, y, p_total=None)")
print("      Compare candidates via CV and EBIC")
print("      candidates: Dict[name -> (Phi, support)]")
print("      Returns: dict with cv_results, ebic_results, best models")
print()
print("  get_best_model(criterion='cv')")
print("      Get best model by 'cv', 'ebic', or 'onese'")
print()
print("  print_comparison_report()")
print("      Print detailed comparison results")
print()
print("Key Parameters:")
print("  n_folds: Number of CV folds (default: 5)")
print("  ebic_gamma: EBIC parameter 0-1 (default: 0.5)")
print()
print("EBIC gamma guidelines:")
print("  0.0 = Near-BIC behavior (moderate p)")
print("  0.5 = Balanced (standard high-dim)")
print("  1.0 = Maximum penalty (p >> n)")
print()
print("Usage Example:")
print("-" * 70)
print("""
# Collect candidate models from Stage 2
candidates = {
    'pysr': (Phi, pysr_support),
    'stlsq': (Phi, stlsq_support),
    'alasso': (Phi, alasso_support)
}

# Compare models
selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
result = selector.compare_models(candidates, y, p_total=Phi.shape[1])

# Get best model
print(f"Best by CV: {result['best_model_cv']}")
print(f"Best by EBIC: {result['best_model_ebic']}")
print(f"Best by One-SE: {result['best_model_onese']}")
""")
print()
print("=" * 70)
print("Module loaded successfully. Import via: %run 09_ModelSelection.ipynb")
print("=" * 70)